In [ ]:
import sys
import re, numpy as np
import pandas
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [ ]:
comments = pandas.read_csv('COMMENTSFILE.csv')
comments.sample(5)

In [ ]:
c = comments[['docket_id', 'subject', 'title', 'gov_agency', 'abstract']]
c.sample(10)

In [ ]:
#TODO: each reg per agency, generate ldavis
fs_df.docket_id.value_counts()

In [ ]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = str(sent)
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = test_df.comment.values.tolist()
data_words = list(sent_to_words(data))

In [ ]:
import re, os, csv 
import pandas as pd 

test_df = pd.read_csv('data/fs_comments.csv')
test_df = test_df.sample(4000)
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=word_tokenize, stop_words='english')

stemmer = PorterStemmer()
test_df["comment"] = test_df['comment'].str.replace('[^\w\s]','')
test_df['stemmed'] = test_df['comment'].apply(lambda x: stemmer.stem(str(x).lower())) 
test_df['stemmed'].sample(10)

In [ ]:
dtm = cv.fit_transform(test_df['stemmed'])

from sklearn.decomposition import LatentDirichletAllocation
# Build LDA Model with GridSearch params
lda_model = LatentDirichletAllocation(n_components=50,
                                      learning_decay=0.5,         
                                      max_iter=50,                
                                      learning_method='online',   
                                      random_state=42,            
                                      batch_size=5000,            
                                      evaluate_every = -1,        
                                      n_jobs = -1)                
lda_output = lda_model.fit_transform(dtm)

In [ ]:
lda_output

In [ ]:
#dtm = cv.fit_transform(test_df['stemmed'])
features = cv.transform(test_df['stemmed'])
features = pd.DataFrame(
    features.todense(), index=test_df.index
)

#print(type(features))

doc_topics = lda_model.transform(features)
topic_matrix = pd.DataFrame(
    doc_topics,
    #10 topics
    columns=["topic_{}".format(i) for i in range(0, 50)],
    index=features.index,
)

topic_matrix.sample(10)

In [ ]:
full_matrix = pd.merge(topic_matrix, test_df[['stemmed', 'comment', 'docket_id']], left_index=True, right_index=True)
full_matrix.sample(10)

In [ ]:
full_matrix.to_csv('fs_new_total.csv')

In [ ]:
import numpy as np
import pandas
def show_topics(vectorizer=cv, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=cv, lda_model=lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pandas.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

In [ ]:
df_topic_keywords.to_csv('fs_new_total_topics.csv')

In [ ]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')